In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import floor

file_pdcp_dl = "/home/luca/Scrivania/TLCnet_prj/ns3-mmwave/output6/mcDlPdcpStats_21_12_2017_01_24_56.txt"

with open(file_pdcp_dl, 'r') as f: 
    in_file_pdcp_dl = f.readlines()
    f.close()

n_ues = 12;
ues_rx = [[] for _ in range(n_ues)]
time_quantum = 0.005
start_time = 0
end_time = 0

for line in in_file_pdcp_dl:
    tokens = line.rstrip().split(' ')
    if tokens[0] == "Rx":
        time = float(tokens[1])
        ue = int(tokens[3]) - 1
        lcid = int(tokens[5])
        pck_size = int(tokens[6])
        pck_delay = int(tokens[7])
        if lcid > 1:
            ues_rx[ue].append((time, pck_size, pck_delay))
            if time > end_time:
                end_time = time
                
times = np.arange(start_time, end_time + time_quantum, time_quantum)
throughput = np.zeros((n_ues, len(times)))
delay = np.zeros((n_ues, len(times)))

for i in range(n_ues):
    count = np.zeros(len(times))
    for pck in ues_rx[i]:
        throughput[i, int(floor(pck[0] / time_quantum))] += pck[1]
        delay[i, int(floor(pck[0] / time_quantum))] += pck[2]
        count[int(floor(pck[0] / time_quantum))] += 1
    count = count.clip(min = 1)
    delay[i] = np.divide(delay[i], count)

throughput /= (10e6 * time_quantum)
mean_throughput = [np.mean(throughput[i]) for i in range(n_ues)]
sigma_throughput = [np.sqrt(np.mean(np.square(throughput[i] - mean_throughput[i]))) for i in range(n_ues)]

delay /= 10e6
mean_delay = [np.mean(delay[i]) for i in range(n_ues)]
sigma_delay = [np.sqrt(np.mean(np.square(delay[i] - mean_delay[i]))) for i in range(n_ues)]

for i in range(n_ues):
    fig = plt.figure(figsize = (20, 10))
    ax1 = plt.gca()
    ax1.set_xlim(start_time, end_time)
    ax1.xaxis.set_ticks(np.arange(start_time, end_time, time_quantum))
    ax1.xaxis.set_ticklabels(np.arange(start_time, end_time, time_quantum))

    plt.subplot(2, 1, 1)
    plt.title("Throughput ue" + str(i + 1))
    plt.xlabel("Time [s]")
    plt.ylabel('Througput [Mb/s]')
    plt.grid(True)

    plt.plot(times, throughput[i], lw = 2, color = 'b')
    plt.plot(times, np.ones(len(times)) * mean_throughput[i], '--', lw = 2, color = 'b')
    plt.plot(times, np.ones(len(times)) * sigma_throughput[i], '--', lw = 2, color = 'g')
    
    plt.legend(['throughput', 'mean', 'sigma'])
    
    plt.subplot(2, 1, 2)
    plt.title("Delay ue" + str(i + 1))
    plt.xlabel("Time [s]")
    plt.ylabel('Delay [ms]')
    plt.grid(True)

    plt.plot(times, delay[i], lw = 2, color = '#800080')
    plt.plot(times, np.ones(len(times)) * mean_delay[i], '--', lw = 2, color = '#800080')
    plt.plot(times, np.ones(len(times)) * sigma_delay[i], '--', lw = 2, color = 'r')
    
    plt.legend(['delay', 'mean', 'sigma'])
    
    plt.tight_layout()
    plt.show()